In [1]:
import glob, sys
import numpy as np
import matplotlib.pyplot as plt
import time
import pyccl as ccl
cosmo = ccl.Cosmology(Omega_c = 0.30711 - 0.048254, Omega_b = 0.048254, h = 0.677, sigma8 = 0.8288, n_s=0.96)

In [2]:
%load_ext autoreload
%autoreload 2
import CL_COUNT_class_cluster_abundance as cl_count

ModuleNotFoundError: No module named 'CL_COUNT_class_cluster_abundance'

initialize the `ClusterAbundance` object

In [ ]:
clc = cl_count.ClusterAbundance()
#choose the halo mass function and mass definition
massdef = ccl.halos.massdef.MassDef('vir', 'critical', c_m_relation=None)
hmd = ccl.halos.hmfunc.MassFuncDespali16(cosmo, mass_def=massdef)
clc.set_cosmology(cosmo = cosmo, hmd = hmd, massdef = massdef)
clc.sky_area = (0.25)*4*np.pi
clc.f_sky = clc.sky_area/4*np.pi

# Binned approach
Compute the 2D integral

## model

$$
N_{\alpha\beta}^{\rm predicted} = \Omega_s\int_{z_\alpha}^{z_{\alpha + 1}} dz\int_{\log_{10}M_\beta}^{\log_{10}M_{\beta + 1}}\frac{d^2V(z)}{dz d\Omega}\frac{dn(M,z)}{d\log_{10}M}d\log_{10}M
$$

define mass & redshift binning

In [ ]:
z_edges = np.linspace(0.25, 1.25, 10)
logm_edges = np.linspace(14, 14.8, 10)

In [ ]:
#define redshift-mass grid
z_grid = np.linspace(0., 3, 300)
logm_grid = np.linspace(12,16, 300)

### Abundance prediction

`grid`

In [ ]:
t0 = time.time()
clc.compute_multiplicity_grid_MZ(z_grid = z_grid, logm_grid = logm_grid)
Abundance_grid = clc.Cluster_Abundance_MZ(zbin_edges = z_edges, proxybin_edges = logm_edges, method = 'grid')
t1 = time.time()
print('time (seconds) = ' + str(t1-t0))
time_simps = t1-t0

### Shot noise

In [ ]:
shot_noise_variance = np.diag(Abundance_grid.flatten())

In [ ]:
covariance = shot_noise_variance #here, we do not take account of sample variance

## data

In [ ]:
N_obs = np.random.multivariate_normal(Abundance_grid.flatten(), covariance)

## Likelihood

In [ ]:
%load_ext autoreload
%autoreload 2
import CL_COUNT_class_likelihood as lnL

In [ ]:
Likelihood = lnL.Likelihood()

### Binned Gaussian

$L = \mathcal{G}(\vec{\widehat{N}}|\vec{\mu}, \Sigma_{SV})$

In [ ]:
lnL_Binned_Gaussian = Likelihood.lnLikelihood_Binned_Gaussian(Abundance_grid, N_obs.reshape(len(z_edges)-1, len(logm_edges)-1), covariance)
print(Likelihood.lnL_Binned_Gaussian)

### Binned Poisson

$L = \prod_i^N\mathcal{P}(\widehat{N}_i| \mu_i)$

In [ ]:
lnL_Binned_Poissonian = Likelihood.lnLikelihood_Binned_Poissonian(Abundance_grid, N_obs.reshape(len(z_edges)-1, len(logm_edges)-1))
print(Likelihood.lnL_Binned_Poissonian)

# Unbinned Approach

## data

In [ ]:
#Generate mass, redshift catalog
n_cluster = 100000
redshift, M200c = np.random.random(n_cluster)*(1 - 0.2) + 0.2, 10**(np.random.random(n_cluster)*(15 - 13) + 13)

## model

$$
\lambda(m_i, z_i) = \Omega_s\frac{d^2V(z_i)}{dz d\Omega}\frac{dn(M_i,z_i)}{d\log_{10}M}
$$

In [ ]:
t0 = time.time()
clc.compute_multiplicity_grid_MZ(z_grid = z_grid, logm_grid = logm_grid)
multiplicity_interp = clc.multiplicity_function_individual_MZ(z = redshift, logm = np.log10(M200c), method = 'interp')
t1 = time.time()
print('time (seconds) = ' + str(t1-t0))

In [ ]:
N_tot = clc.Cluster_Abundance_MZ(zbin_edges = [.2, 1], proxybin_edges = [13,15], method = 'grid')[0][0]

## Likelihood

$\log L = \sum_i^N \log\lambda(m_i, z_i) - N_{tot}$

In [ ]:
Likelihood.lnLikelihood_UnBinned_Poissonian(multiplicity_interp, N_tot)
print(Likelihood.lnL_UnBinned_Poissonian)